Initial Imports:

In [1]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import Normalize
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler

1.***[Loading the data]***:
For MNIST:
In Python using TensorFlow and Keras:

In [2]:
# Transformations to apply to the dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the data
])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

For CIFAR-10:


In [3]:
# Transformations for CIFAR-10
transform_cifar = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the data
])

# Load the CIFAR-10 dataset
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)


Files already downloaded and verified
Files already downloaded and verified


Paper results:

In [4]:
file_path = 'paper_results.csv'

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(file_path)
print("Results obtained in the paper:")
data

Results obtained in the paper:


,Method,MNIST ACC,MNIST NMI,USPS ACC,USPS NMI,CIFAR-10 ACC,CIFAR-10 NMI
0,K-means,0.58,0.49,0.48,0.42,0.14,0.12
1,Deep Cluster,0.86,0.83,0.67,0.69,NA,NA
2,Deep K-means,0.84,0.80,0.76,0.78,NA,NA
3,CSC No Flatten,0.85,0.79,0.83,0.78,0.12,0.08
4,CSC No Filter,0.83,0.76,0.84,0.79,0.14,0.10
5,CSC No Voting,0.82,0.77,0.82,0.76,0.14,0.10
6,CSC,0.86,0.81,0.83,0.79,0.15,0.11


### A - 1) Normalization

In [3]:
# Data normalization with min-max method
def min_max_scale_dataset(dataset, batch_size=64):
    
    min_pixel_value = float('inf')
    max_pixel_value = float('-inf')

    for images, _ in DataLoader(dataset, batch_size=batch_size, shuffle=True):
        min_pixel_value = min(min_pixel_value, images.min())
        max_pixel_value = max(max_pixel_value, images.max())

    
    min_max_scaler = Normalize(min_pixel_value, max_pixel_value)
    # Apply the min-max scaling to the dataset
    transform = transforms.Compose([transforms.ToTensor(), min_max_scaler])
    normalized_dataset = MNIST(root='./data', train=True, download=True, transform=transform)

    normalized_loader = DataLoader(normalized_dataset, batch_size=batch_size, shuffle=True)

    return normalized_loader

In [4]:
BATCH_SIZE = 64

In [5]:
mnist_train_loader = min_max_scale_dataset(mnist_trainset)
mnist_test_loader = min_max_scale_dataset(mnist_testset)

### A - 2) Autoencoder

The autoencoder architecture

In [6]:
#Defining the model
class ConvAutoencoder(nn.Module):
    def __init__(self, input_channels):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, input_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

        self.bottleneck = nn.Linear(16 * 14 * 14, 500)

    def forward(self, x):
        x = self.encoder(x)
        flatten_x = x.view(x.size(0), -1)  # Flatten the output from the encoder
        bottleneck_features = self.bottleneck(flatten_x)
        reconstructed = self.decoder(x)
        return bottleneck_features, reconstructed

In [7]:
input_channels_mnist = 1  # MNIST images are grayscale
input_channels_cifar = 3  # CIFAR-10 images have 3 channels (RGB)

mnist_autoencoder = ConvAutoencoder(input_channels_mnist)
cifar_autoencoder = ConvAutoencoder(input_channels_cifar)

In [8]:
criterion = nn.MSELoss()
optimizer_mnist = torch.optim.Adam(mnist_autoencoder.parameters(), lr=0.001)
optimizer_cifar = torch.optim.Adam(cifar_autoencoder.parameters(), lr=0.001)

The loop for training the model

In [9]:
NUM_EPOCH = 10
def training_loop (model, loader, optimizer):
    model.train()

    for epoch in range(NUM_EPOCH):
        
        for data in loader:
            img, _ = data
            optimizer.zero_grad()
            _, output = model(img)
            loss = criterion(output, img)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{NUM_EPOCH}], Loss: {loss.item():.4f}')

    return model

In [10]:
mnist_autoencoder = training_loop(mnist_autoencoder,mnist_train_loader,optimizer_mnist)
#cifar_autoencoder = training_loop(cifar_autoencoder,cifar_train_loader,optimizer_cifar)

Epoch [1/10], Loss: 0.0021
Epoch [2/10], Loss: 0.0011
Epoch [3/10], Loss: 0.0008
Epoch [4/10], Loss: 0.0006
Epoch [5/10], Loss: 0.0006
Epoch [6/10], Loss: 0.0005
Epoch [7/10], Loss: 0.0005
Epoch [8/10], Loss: 0.0004
Epoch [9/10], Loss: 0.0004
Epoch [10/10], Loss: 0.0004


The evaluation loop to extract 500 features from the bottleneck layer for each input image

In [11]:
def evaluation (model, loader):
    '''
    output shape : bottelneck_feature (nb img,500)
    '''
    model.eval()

    bottleneck_features_array = []
    outputs_array = []
    total_loss = 0
 
    for data in loader:
        img, _ = data
       
        bottleneck_features, output = model(img)

        for features in bottleneck_features:
            bottleneck_features_array.append(features.detach().numpy())
        outputs_array.append(output.detach().numpy())

        loss = criterion(output, img)
        total_loss += loss.item()

    average_loss = total_loss / len(loader)
    print("Evaluation loss : ",average_loss)
    
    return np.array(bottleneck_features_array), outputs_array


### B - Best features selection

Do the Non negativ Matrix Factorization (NMF) to estimate a decomposition W*H from V \
Calcul the error rate E from W*H+E = V \
Sort feature by error rate and keep the 50% best

In [19]:
def feature_filtering_nmf(features, k=1, removal_percentage=50):
    '''
    output shape : (nb img,250)
    '''

    #it doesn't work because of negative value so we remove the negative here
    min_value = np.min(features)
    features_shifted = features - min_value + 1e-10 

    # NMF : find the decomposition V=W*H
    nmf_model = NMF(n_components=k, init='random', random_state=None)
    nmf_features = nmf_model.fit_transform(features_shifted)

    #reconstruc W*H to evaluate with V to find E
    reconstructed_features = np.dot(nmf_features, nmf_model.components_)
    reconstruction_error = np.abs(features - reconstructed_features)

    # Sort features by their E
    sorted_indices = np.argsort(reconstruction_error, axis=1)
    num_features_to_keep = int((100 - removal_percentage) / 100 * features.shape[1])
    selected_features = features[np.arange(features.shape[0])[:, None], sorted_indices[:, :num_features_to_keep]]
    scaler = StandardScaler()
    standardized_features = scaler.fit_transform(selected_features)

    return np.array(standardized_features)

### Loop over A and B

do a loop over steps A and B  to increase number of features

In [25]:
features_matrix = None

for i in range(10):
    features,_ = evaluation(mnist_autoencoder,mnist_test_loader)
    sorted_features = feature_filtering_nmf(features)
    
    if i == 0 :
        features_matrix = sorted_features
    else :
        features_matrix = np.concatenate((features_matrix,sorted_features),axis=1)

Evaluation loss :  0.00038035106989167835
Evaluation loss :  0.000380320429360357
Evaluation loss :  0.000380341081351572
Evaluation loss :  0.0003803369247784981
Evaluation loss :  0.0003803375105163567
Evaluation loss :  0.0003803434074860809
Evaluation loss :  0.0003803311711559986
Evaluation loss :  0.0003803368395458518
Evaluation loss :  0.00038034402512024835
Evaluation loss :  0.00038035992083813885


In [28]:
print(features_matrix.shape)

#C'est cette matrice qui doit être utilisé pour l'étape C
# sa dimensio est (60000,2500)
# 60 000 c'est le nombre d'image passée en input à l'origine
# 2500 c'est le nombre de features décrivant chaque image

(60000, 2500)


### C - Variational Autoencoder

# Load MNIST and CIFAR-10 datasets

In [ ]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
cifar_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified
